In [ ]:
#Importing supporting libraries
from selenium import webdriver
import datetime
import time
import pandas as pd
from bs4 import BeautifulSoup
import math
import datetime
import requests
import os

#Starting Chrome Incognito window to automatically login.
option = webdriver.ChromeOptions()
option.add_argument("incognito")
option.add_argument("--disable-notifications")
#option.add_argument("--headless")
option.add_argument("--start-maximized");
driver = webdriver.Chrome(executable_path='./chromedriver', options=option)
time.sleep(1)


category_urls = []
product_urls = []

names = []
skus = []
bar_codes = []


source = requests.get("https://www.markuponline.com/")
soup = BeautifulSoup(source.content, "html.parser")
category_links = soup.find_all("a", {"id": "catclass"})

for category_link in category_links:
    category_url = category_link.get("href")
    category_urls.append(category_url)
    
    
for index, category_url in enumerate(category_urls):
    
    print(f"Fetching Category: {str(index)}")
    driver.get(category_url)
    time.sleep(4)
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    lis = soup.find("ul", {"class": "lzd-site-menu-grand"}).find_all("li", {"class": "item"})
    for li in lis:
        product_url = li.find("a").get("href")
        product_urls.append(product_url)
        
        
for index, product_url in enumerate(product_urls):
    
    print(f"Fetching Product: {str(index)}")
    source = requests.get(product_url)
    soup = BeautifulSoup(source.content, "html.parser")
    
    try:
        name = soup.find("div", {"class": "product-shop"}).find("h1").get_text().strip()
        print(name)
        names.append(name)
    except Exception as e:
        print(f"Got exception in names: {str(e)}")
        names.append("")
        
    try:
        sku = soup.find("div", {"class": "stock_des"}).find_all("p")[0].get_text().replace("Product Code:", "").strip()
        print(sku)
        skus.append(sku)
    except Exception as e:
        print(f"Got exception in skus: {str(e)}")
        skus.append("")
        
    try:
        bar_code = soup.find("div", {"class": "stock_des"}).find_all("p")[1].get_text().replace("Barcode:", "").strip()
        print(bar_code)
        bar_codes.append(bar_code)
    except Exception as e:
        print(f"Got exception in bar codes: {str(e)}")
        bar_codes.append("")
        
        
file_name = 'data_{}.xlsx'.format(datetime.datetime.now().strftime("%d%m%Y-%H%M%S"))

output = pd.DataFrame([])
output["Name"] = names
output["SKU"] = skus
output["Bar Code"] = bar_codes
output["URL"] = product_urls
output.to_excel(file_name, index=False)
print("I am done please check the files in the directory💯💯💯")